### This is designed to merge the single papers together to assemble one bigger data-csv that we can at some point use for model training

TODO: 
Investigate: 
https://www.nature.com/articles/s41598-023-41443-4?fromPaywallRec=false 
https://www.nature.com/articles/s41593-023-01514-1.pdf 


# General Notes: 
- To execute the notebook, you frist need to produce the .csv from the underlying papers
- I try to convert and select for ENSG gene rows as we got  datasets from Flo with 60k entries which most probably will be me mayjority


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
from utils.helper import manhattanplot, violinplot_overall, scatter_plot, get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts, convert_gene_names_to_ensembl, clean_age
BASE_PATH="/work/transcriptomics_norm_model"


In [ ]:
### Wimmers 2023 (NOT INCLUDED)
#NOTE This is 1 of 2 Datasets from this paper. 
#NOTE Also this datasets should be clarified with Flo first. It´s PBMC but not clear from whom ? (maybe GSE239799 ??? but also different)
wimmers_2_df = pd.read_csv(f'{BASE_PATH}/2023_wimmers/2023_Wimmers_combi_df.csv')
print(wimmers_2_df.shape)
wimmers_2_df.head()
PLATFORM_GEO_ID_wimmers_23 = "GPL24676"
PLATFORM_DESCRIPTION_wimmers_23 = "Illumina NovaSeq 6000 (Homo sapiens)"
METHOD_wimmers_23 = "RNAseq"
#TODO in the data itself PBMC is mentioned but https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787 it is whole blood 
TYPE_wimmers_23 = "blood"
study_accession_wimmers_23 = "???"
desease_wimmers_23 = "SARS-CoV-2 / healthy"

In [ ]:
### Wimmers 2023 
#NOTE This is 2 of 2 Datasets from this paper. 
wimmers_GEO239787_df = pd.read_csv(PATH_WIMMER2023, index_col=0)
print(wimmers_GEO239787_df.shape)
wimmers_GEO239787_df = wimmers_GEO239787_df.rename(columns={
  'ID':'subject_accession', 
  'geo_accession':'data_accession',
  'characteristics_ch1.4':'age',
  'characteristics_ch1.3':'gender',
  'Status':'desease', 
  'contact_city':'origin',
  'library_strategy':'METHOD', 
  'source_name_ch1':'TYPE',
  'platform_id':'PLATFORM_GEO_ID',
  'instrument_model':'PLATFORM_DESCRIPTION',
})
wimmers_GEO239787_df['study_accession']='GSE239787'
wimmers_GEO239787_df['origin']='Contact City: Palo Alto'
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age'].str.replace('age, months: ', '').astype(float)
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age']/12

wimmers_GEO239787_df_reduced = reorder_columns_by_metadata_and_gene_counts(wimmers_GEO239787_df)

print(wimmers_GEO239787_df_reduced.shape)
wimmers_GEO239787_df_reduced.tail()

In [ ]:
# 2020 Arunachalam, Wimmers
#NOTE migh had some issues in the prep df file, but the GEO actually did a good job 
#NOTE I can probably add 4 more subjects but want to clarify with flo first
aranachalam_2020_GSE152418_df = pd.read_csv(PATH_ARUNACHALAM2020_FINALDESTINATION, index_col=0)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df = aranachalam_2020_GSE152418_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'Age':'age',
  'characteristics_ch1.2':'gender',
  'characteristics_ch1.3':'desease', 
  'contact_city':'origin',
  'library_strategy':'METHOD', 
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2020_GSE152418_df['study_accession'] = 'GSE152418'
aranachalam_2020_GSE152418_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2020_GSE152418_df)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df.head()

In [ ]:
# 2021 Thair COVID 
# NOTE 

thair_2021_GSE152641_df = pd.read_csv(f"{BASE_PATH}/2021_thair/thair_combi_covid.csv", low_memory=False, index_col=0)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.loc[1:86, :] # there are 3 meta rows in the csv... 
print(thair_2021_GSE152641_df.shape)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.rename(columns={
  'title':'subject_accession', 
    'geo_accession':'data_accession',
    'characteristics_ch1.3':'age',
    'characteristics_ch1.4':'gender',
    'characteristics_ch1':'desease',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'characteristics_ch1.2':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
thair_2021_GSE152641_df['study_accession'] ='GSE152641'
thair_2021_GSE152641_ordered_df = reorder_columns_by_metadata_and_gene_counts(thair_2021_GSE152641_df)
print(thair_2021_GSE152641_ordered_df.shape)
thair_2021_GSE152641_ordered_df.head()

In [ ]:
# 2021 Aranachalam VACCINE
#NOTE Had to convert Gene names to ENSEMBL which sometimes results in a not found version -> Flo how to handle 
#NOTE Vaccine study, i just took the Baseline (blood before the vaccine was given)

import copy

aranachalam_2021_GSE169159_df = pd.read_csv(f'{BASE_PATH}/2021_arunachalam/2021_aranachalam_vaccine_combi_df_baseline.csv')
print(aranachalam_2021_GSE169159_df.shape)
aranachalam_2021_GSE169159_df = aranachalam_2021_GSE169159_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'age':'age',
    'sex':'gender',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2021_GSE169159_df['desease']='Healthy'
aranachalam_2021_GSE169159_df['study_accession']='GSE169159'

## This is weird as the overwrite of a subset of columns of a dataframe seems not to work. I tried stuff in different patterns here
print(f"First gene: {aranachalam_2021_GSE169159_df.columns.get_loc('FGR')}")
gene_names_to_convert = aranachalam_2021_GSE169159_df.iloc[:, 38:-2].copy().columns
aranachalam_2021_GSE169159_ordered_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2021_GSE169159_df, 
                                                                                    metadata_cols=METADATA_COLS,
                                                                                    gene_columns=gene_names_to_convert)
print(f"First gene: {aranachalam_2021_GSE169159_ordered_df.columns.get_loc('FGR')}")
converted_gene_names = convert_gene_names_to_ensembl(aranachalam_2021_GSE169159_ordered_df.iloc[:, 11:].copy().columns)

# Now i want to overwrite every column of the dataset (also i dont need to use loc or iloc)
all_dataset_new_columns = METADATA_COLS + converted_gene_names
aranachalam_2021_GSE169159_ordered_df.columns = all_dataset_new_columns

print(aranachalam_2021_GSE169159_ordered_df.columns[60])
print(converted_gene_names[49]) # as i need to add the meta cols ... 

print(aranachalam_2021_GSE169159_ordered_df.shape)
aranachalam_2021_GSE169159_ordered_df.tail()

In [ ]:
# 2023 feargal
# '/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv'
feargal_2023_GSE199750_df_init = pd.read_csv('/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv')
print(feargal_2023_GSE199750_df_init.shape)
# HERE IS A BUG, the data got age and gender cols confused in some rows... 
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.2'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3']
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.4']

In [ ]:

feargal_2023_GSE199750_df_init['characteristics_ch1.3'].unique()


In [ ]:

feargal_2023_GSE199750_df = feargal_2023_GSE199750_df_init.rename(columns={
        'characteristics_ch1':'subject_accession', 
           'geo_accession':'data_accession',
           'characteristics_ch1.2':'age',
          'characteristics_ch1.3':'gender',
          'contact_city':'origin',
           'library_strategy':'METHOD',
           'source_name_ch1':'TYPE',
          'instrument_model':'PLATFORM_DESCRIPTION',
          'platform_id':'PLATFORM_GEO_ID',
})
feargal_2023_GSE199750_df['desease']='Healthy'
feargal_2023_GSE199750_df['study_accession']='GSE199750'
feargal_2023_GSE199750_reordered_df = reorder_columns_by_metadata_and_gene_counts(feargal_2023_GSE199750_df)
print(feargal_2023_GSE199750_reordered_df.shape)
feargal_2023_GSE199750_reordered_df.head()

In [ ]:
#SOARES-SCHANINSKI
#NOTE CHIKV virus patients and ~20 healthy controls

soares_df = pd.read_csv('/work/transcriptomics_norm_model/2019_soares/2019_soares_combi_df.csv')
print(soares_df.shape)
soares_df = soares_df.rename(columns={
                'BioSample':'subject_accession', 
                'Run':'data_accession',
                'Age':'age',
                'sex':'gender',
                'Center Name':'origin',
                'Assay Type':'METHOD',
                'tissue':'TYPE',
                'Instrument':'PLATFORM_DESCRIPTION',
                'disease': 'desease'
})
soares_df['study_accession']='BioProject'
soares_df['PLATFORM_GEO_ID']='-'
soares_df = reorder_columns_by_metadata_and_gene_counts(soares_df)
print(soares_df.shape)
soares_df.head()

# 2023 feargal
# '/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv'
feargal_2023_GSE199750_df_init = pd.read_csv('/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv')
print(feargal_2023_GSE199750_df_init.shape)
# HERE IS A BUG, the data got age and gender cols confused in some rows... 
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.2'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3']
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.4']

In [ ]:
concatinated_df = pd.concat([aranachalam_2020_GSE152418_df, 
                             wimmers_GEO239787_df_reduced, 
                             thair_2021_GSE152641_df, 
                             aranachalam_2021_GSE169159_ordered_df, 
                             feargal_2023_GSE199750_reordered_df,
                             soares_df])
print(concatinated_df.shape)
concatinated_df.head()

### Unify the different valuee 

In [ ]:
mapping = {
    'disease state: COVID-19': 'Sars-CoV-2',
    'disease: COVID19': 'Sars-CoV-2',
    'acute': 'Sars-CoV-2',
    'acute-omicron': 'Sars-CoV-2',
    'disease state: Healthy': 'Healthy',
    'healthy': 'Healthy',
    'disease: Healthy control': 'Healthy',
    'Healthy': 'Healthy',
    'Control': 'Healthy',
}
concatinated_df['desease'] = concatinated_df['desease'].replace(mapping)
concatinated_df['desease'].unique()

In [ ]:
mapping = {
    'gender: F': 'Female',
    'gender: M': 'Male',
    'Sex: Male': 'Male',
    'Sex: Female': 'Female',
    'female': 'Female',
    'male': 'Male',
}
concatinated_df['gender'] = concatinated_df['gender'].replace(mapping)
concatinated_df['gender'].unique()


In [ ]:

# Apply the function and convert column to float
concatinated_df['age'] = concatinated_df['age'].apply(clean_age).astype(float)


### Unify the different valuee 

In [ ]:
mapping = {
    'disease state: COVID-19': 'Sars-CoV-2',
    'disease: COVID19': 'Sars-CoV-2',
    'acute': 'Sars-CoV-2',
    'acute-omicron': 'Sars-CoV-2',
    'disease state: Healthy': 'Healthy',
    'healthy': 'Healthy',
    'disease: Healthy control': 'Healthy',
    'Healthy': 'Healthy',
    'Control': 'Healthy',
}
concatinated_df['desease'] = concatinated_df['desease'].replace(mapping)
concatinated_df['desease'].unique()

In [ ]:
mapping = {
    'gender: F': 'Female',
    'gender: M': 'Male',
    'Sex: Male': 'Male',
    'Sex: Female': 'Female',
    'female': 'Female',
    'male': 'Male',
}
concatinated_df['gender'] = concatinated_df['gender'].replace(mapping)
concatinated_df['gender'].unique()


# Why is there nan

In [ ]:

# Apply the function and convert column to float
concatinated_df['age'] = concatinated_df['age'].apply(clean_age).astype(float)

In [ ]:
#Filter for duplicated subjects. Here just the same subj ID is most likely not sufficient (check those "draws and so on ")
print(len(concatinated_df['subject_accession'].unique()))
concatinated_df_per_subject = concatinated_df.drop_duplicates(subset=['subject_accession'])
concatinated_df_per_subject = concatinated_df_per_subject[~concatinated_df_per_subject['subject_accession'].str.endswith('Draw-2', na=False)]
print(concatinated_df_per_subject.shape)


In [ ]:
#NOTE: Healthy
#TODO do we really lose 25 subjects for nan values in age and gender ?
concatinated_df_per_subject_healthy = concatinated_df_per_subject[concatinated_df_per_subject['desease'] == 'Healthy']
print(concatinated_df_per_subject_healthy.shape)
concatinated_df_per_subject_healthy = concatinated_df_per_subject_healthy.dropna(subset=['age', 'gender', 'desease'])
print(concatinated_df_per_subject_healthy.shape)
concatinated_df_per_subject_healthy
violinplot_overall(concatinated_df_per_subject_healthy, 'gender', 'age', hue_split='gender')

In [ ]:
concatinated_df_per_subject[concatinated_df_per_subject['age']<0]

In [ ]:
#NOTE: Sars-CoV-2
#TODO do we really lose 25 subjects for nan values in age and gender ?
concatinated_df_per_subject_covid = concatinated_df_per_subject[concatinated_df_per_subject['desease'] == 'Sars-CoV-2']
print(concatinated_df_per_subject_covid.shape)
concatinated_df_per_subject_covid = concatinated_df_per_subject_covid.dropna(subset=['age', 'gender', 'desease'])
print(concatinated_df_per_subject_covid.shape)
concatinated_df_per_subject_covid
violinplot_overall(concatinated_df_per_subject_covid, 'gender', 'age', hue_split='gender')

### GTEx database

In [ ]:


def read_gct(file_path):
    with open(file_path, 'r') as file:
        # Skip the first two lines of metadata
        version = file.readline().strip()
        dimensions = file.readline().strip()
        
        # Read the rest of the file into a DataFrame
        df = pd.read_csv(file, sep='\t')
    
    return version, dimensions, df

df = read_gct("/home/eliasschreiner/PROJEKT/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct")
#print(df.shape)


In [ ]:
df[2].shape
GTEx_df = df[2].T
print(GTEx_df.shape)
GTEx_df.head()

In [ ]:
'''As metadata we get sth like this: 
SUBJID	SEX	AGE	DTHHRDY
GTEX-1117F	2	60-69	4
GTEX-111CU	1	50-59	0
GTEX-111FC	1	60-69	1
GTEX-111VG	1	60-69	3
GTEX-111YS	1	60-69	0
GTEX-1122O	2	60-69	0
GTEX-1128S	2	60-69	2 '''

### Unify the different valuee 

In [ ]:
mapping = {
    'disease state: COVID-19': 'Sars-CoV-2',
    'disease: COVID19': 'Sars-CoV-2',
    'acute': 'Sars-CoV-2',
    'acute-omicron': 'Sars-CoV-2',
    'disease state: Healthy': 'Healthy',
    'healthy': 'Healthy',
    'disease: Healthy control': 'Healthy',
    'Healthy': 'Healthy',
}
concatinated_df['desease'] = concatinated_df['desease'].replace(mapping)
concatinated_df['desease'].unique()

In [ ]:
mapping = {
    'gender: F': 'Female',
    'gender: M': 'Male',
    'Sex: Male': 'Male',
    'Sex: Female': 'Female',
}
concatinated_df['gender'] = concatinated_df['gender'].replace(mapping)
concatinated_df['gender'].unique()


In [ ]:

# Apply the function and convert column to float
concatinated_df['age'] = concatinated_df['age'].apply(clean_age).astype(float)
